Kaggle Challenge Data Analytics (Supervised regression task) Python Code Key Steps
============================================

Overall, there are four parts of this analysis:
1. Data Exploratory; 
2. Data Engineering; 
3. Finalizing Training Dataset; 
4. Modeling and Evaluation

Assumptions for the analysis:<br>
1. Assume measurement values less than or equal to zero are invalid;
2. Assume there is start up period of the process, which last for 60s;
3. Assume only focus on raw material properties and controlled variables based on project goal.


[1. Data Exploratory](#1.Data Exploratory)
--------------------
Have an overview of the dataset, get the idea what to do in next step Data Engineering.

[2. Data Engineering](#2.Data Engineering)
--------------------
[2.1 Missing value](#Missing value):<br> There's no missing values.<br>
[2.2 Outlier](#Outlier):<br> Remove invalid data points, including measurement values less than or equal to zero, values might be collected during the start up period (assume first 60s), abnormal changed values. And then conduct [moving average (30s window)](#smooth noise) to remove random noise.<br>
[2.3 Variable type](#Variable type): <br> Variables are all continous numerical data, so there is no need to do extra processing.<br>
[2.4 Scaling](#Scaling): <br> Although normalizing features is a general requirement for many machine learning algorithms, it's inproper to do normalization without background knowledge of input variables. Therefore, input variables are not normalized.<br>
[2.5 Multi-collinearity](#Multi-collinearity): <br> There is severe multi-collinearity problem among the input variables for stage1 and stage2, especially for material property variables, which should be noticed while modeling.<br>
Rule of thumb: VIF >10, high multi-collinearity<br>
[2.6 Feature transformation](#Feature transformation): <br> Not be able to do feature transformation (generate new meaningful features) without background knowledge of input variables.

[3. Finalizing Training Dataset](#3.Finalizing Training Dataset)
--------------------
Prepare training dataset for modeling:<br>
Input variables: <br>
Stage 1:<br>
Machine1.RawMaterial.Property1<br>
Machine1.RawMaterial.Property2<br>
Machine1.RawMaterial.Property3<br>
Machine1.RawMaterial.Property4<br>
Machine1.Zone1Temperature.C.Actual<br>
Machine1.Zone2Temperature.C.Actual<br>
Machine1.MotorRPM.C.Actual<br>
Machine1.ExitZoneTemperature.C.Actual<br>
Machine2.RawMaterial.Property1<br>
Machine2.RawMaterial.Property2<br>
Machine2.RawMaterial.Property3<br>
Machine2.RawMaterial.Property4<br>
Machine2.Zone1Temperature.C.Actual<br>
Machine2.Zone2Temperature.C.Actual<br>
Machine2.MotorRPM.C.Actual<br>
Machine2.ExitZoneTemperature.C.Actual<br>
Machine3.RawMaterial.Property1<br>
Machine3.RawMaterial.Property2<br>
Machine3.RawMaterial.Property3<br>
Machine3.RawMaterial.Property4<br>
Machine3.Zone1Temperature.C.Actual<br>
Machine3.Zone2Temperature.C.Actual<br>
Machine3.MotorRPM.C.Actual<br>
Machine3.ExitZoneTemperature.C.Actual<br>
FirstStage.CombinerOperation.Temperature3.C.Actual<br>

Stage 2:
Machine1.RawMaterial.Property1<br>
Machine1.RawMaterial.Property2<br>
Machine1.RawMaterial.Property3<br>
Machine1.RawMaterial.Property4<br>
Machine1.Zone1Temperature.C.Actual<br>
Machine1.Zone2Temperature.C.Actual<br>
Machine1.MotorRPM.C.Actual<br>
Machine1.ExitZoneTemperature.C.Actual<br>
Machine2.RawMaterial.Property1<br>
Machine2.RawMaterial.Property2<br>
Machine2.RawMaterial.Property3<br>
Machine2.RawMaterial.Property4<br>
Machine2.Zone1Temperature.C.Actual<br>
Machine2.Zone2Temperature.C.Actual<br>
Machine2.MotorRPM.C.Actual<br>
Machine2.ExitZoneTemperature.C.Actual<br>
Machine3.RawMaterial.Property1<br>
Machine3.RawMaterial.Property2<br>
Machine3.RawMaterial.Property3<br>
Machine3.RawMaterial.Property4<br>
Machine3.Zone1Temperature.C.Actual<br>
Machine3.Zone2Temperature.C.Actual<br>
Machine3.MotorRPM.C.Actual<br>
Machine3.ExitZoneTemperature.C.Actual<br> 
FirstStage.CombinerOperation.Temperature3.C.Actual  
Machine4.Temperature1.C.Actual  
Machine4.Temperature2.C.Actual  
Machine4.Pressure.C.Actual  
Machine4.Temperature4.C.Actual  
Machine4.Temperature5.C.Actual<br>
Machine5.Temperature1.C.Actual<br> 
Machine5.Temperature2.C.Actual  
Machine5.Temperature3.C.Actual  
Machine5.Temperature4.C.Actual<br> 
Machine5.Temperature5.C.Actual<br> 
Machine5.Temperature6.C.Actual  


Output variables:<br>
Stage1.Measurement%d.DV_ErrorRate or Stage1.Output.Measurement%d.U.Actual <br>
Stage2.Measurement%d.DV_ErrorRate or Stage2.Output.Measurement%d.U.Actual

[4. Modeling and Evaluation](#4.Modeling and Evaluation)
--------------------
Including model building and model performance evaluation. <br>

4.1 Model building:  <br>
Linear Regression and Bayesian Ridge Regression are selected for modeling. Since input variables with narrow range might not provide enough information to model building, complicated models have not been applied.

4.2 Model performance evaluation:  <br>
(1) Mean Absolute Error, Mean Squared Error, Root Mean Squared Error are used to evaluate model performance.<br>
(2) Performance of Bayesian Ridge Regression model is similar with performance of Linear Regression model.<br>
(3) Model performance differ from measurement to measurement, and from stage to stage (more details refer to corresponding bar charts of [Linear Regression Model Performance](#lr performance) and [Bayesian Ridge Regression Model Performance](#brr performance)<br>
(4) Bayesian Ridge Regression model give shrinkage to those coefficients of high VIF variables. [Take Measurement0 at Stage1 for example](#Coefficients).

#### Import Package

In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns

from scipy import stats

from statistics import mode

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn import metrics

from sklearn.externals import joblib

%matplotlib inline
plt.style.use("seaborn-bright")

In [ ]:
# Customized functions for data processing
from DataProcessFunction import error_rate
from DataProcessFunction import clean_outlier
from DataProcessFunction import vif_cal

#### Load data

In [ ]:
RawData = pd.read_csv('RawData.csv')

<a id='1.Data Exploratory'></a>
### 1.Data Exploratory
Have a overview of the data

In [ ]:
Data = RawData # Save a copy before editing to prevent changing the original file

In [ ]:
Data.describe()

#### 1.1 Categorize columns

In [ ]:
Data.columns

In [ ]:
Data['ID'] = Data.reset_index().index # Add rows' ID

In [ ]:
AmbientConditions_cols = [col for col in Data.columns if 'AmbientConditions' in col]
Machine1_cols = [col for col in Data.columns if 'Machine1' in col]
Machine2_cols = [col for col in Data.columns if 'Machine2' in col]
Machine3_cols = [col for col in Data.columns if 'Machine3' in col]
Combiner_cols = [col for col in Data.columns if 'Combiner' in col]
Stage1_cols = [col for col in Data.columns if 'Stage1' in col]
Machine4_cols = [col for col in Data.columns if 'Machine4' in col]
Machine5_cols = [col for col in Data.columns if 'Machine5' in col]
Stage2_cols = [col for col in Data.columns if 'Stage2' in col]

In [ ]:
AmbientConditions_cols

In [ ]:
Machine1_cols

In [ ]:
Combiner_cols

In [ ]:
Stage1_cols

In [ ]:
Machine4_cols

In [ ]:
Stage2_cols

#### 1.2 Check Timestamp

In [ ]:
Data[["time_stamp"]] # Check sample rate

In [ ]:
type(Data["time_stamp"]) # Check column type, shows not the real Timestamp type

#### 1.3 Define input variables
Based on project goal and process features, select raw material properties and controlled variables as input variables

In [ ]:
Machine1_C_cols = [col for col in Machine1_cols if '.U.' not in col]
Machine2_C_cols = [col for col in Machine2_cols if '.U.' not in col]
Machine3_C_cols = [col for col in Machine3_cols if '.U.' not in col]
Combiner_C_cols = [col for col in Combiner_cols if '.U.' not in col]
Machine4_C_cols = [col for col in Machine4_cols if '.U.' not in col]
Machine5_C_cols = [col for col in Machine5_cols if '.U.' not in col]

In [ ]:
C_cols_Stage1 = Machine1_C_cols+Machine2_C_cols+Machine3_C_cols+Combiner_C_cols

In [ ]:
C_cols_Stage2 = Machine1_C_cols+Machine2_C_cols+Machine3_C_cols+Combiner_C_cols+Machine4_C_cols+Machine5_C_cols

#### 1.4 Define output variables
Based on project goal, use error versus setpoint as dependent variables

In [ ]:
Data[Stage1_cols].describe()

In [ ]:
Data[Stage2_cols].describe()

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.loc[:, 'Stage1.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual')
    ax[i].plot(Data.loc[:, 'Stage1.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle='-', label='Setpoint')
    ax[i].set_ylabel('Stage1.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');

# Have a overview of Stage1.Outputs 

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.loc[:, 'Stage2.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual')
    ax[i].plot(Data.loc[:, 'Stage2.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle='-', label='Setpoint')
    ax[i].set_ylabel('Stage2.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');

# Have a overview of Stage2.Outputs

##### Note1: Summary from the output measurement plots above

1.Most of the setpoint values at stage1 are constant, except several ones(0) around 4000s. If these points are outliers due to measurement error, they should be removed from training dataset. Also, constant setpoint values would avoid ratio (error versus setpoints) issues while training models.

2.Some of the actual measurement values are equal or less than 0, if these points are outliers due to measurement error or start up period, they should be removed from training dataset.

3.Several stage1 actual values around 10000s changed suddenly in all 15 measurements. Assumed these points are outliers due to measurement error, and they should be removed from training dataset.

4.There is not obvious trend from the plots. Time elements would not be considered while modeling for temporary.

##### Use error versus setpoint as dependent variables 
(final value rounded to 3 decimals)

In [ ]:
for i in range(15):
    Data["Stage1.Measurement%d.ErrorRate"%(i)]= error_rate(Data,"Stage1.Output.Measurement%d.U.Actual"%(i),"Stage1.Output.Measurement%d.U.Setpoint"%(i))
    Data["Stage2.Measurement%d.ErrorRate"%(i)]= error_rate(Data,"Stage2.Output.Measurement%d.U.Actual"%(i),"Stage2.Output.Measurement%d.U.Setpoint"%(i))

<a id='2.Data Engineering'></a>
### 2. Data Engineering
Dealing with missing value, outlier, variable type, scaling, multi-collinearity, feature transformation

<a id='Missing value'></a>
#### 2.1 Check Missing value

In [ ]:
Original_cols = [col for col in Data.columns if 'ErrorRate' not in col]
for col in Original_cols:
    if Data[col].isna().sum()>0:
        print(col+":%d"%(Data[col].isna().sum()))
# No missing value

<a id='Outlier'></a>
#### 2.2 Check Outlier
Refer to Note1, remove invalid rows from traning set for specific output.

In [ ]:
# Use dictionary to store training data set for each measurement
Data.stage1 = {} 
Data.stage2 = {} 

# Refer to Note1.2, assumed output values less than or equal to 0 are invalid datapoints
for i in range(15):
    Data.stage1["Measurement%d"%(i)] = clean_outlier(Data,"Stage1.Output.Measurement%d.U.Actual"%(i))
    Data.stage2["Measurement%d"%(i)] = clean_outlier(Data,"Stage2.Output.Measurement%d.U.Actual"%(i))
    

In [ ]:
# Refer to Note1.1,get corresponding row ID 
invalid_ID_1 = list(Data[Data['Stage1.Output.Measurement0.U.Setpoint'] < mode(Data['Stage1.Output.Measurement0.U.Setpoint'])].index)

In [ ]:
# Refer to Note1.3,get corresponding row ID 
invalid_ID_2 = list(Data[Data['Stage1.Output.Measurement0.U.Actual'] > mode(Data['Stage1.Output.Measurement0.U.Setpoint'])].index)

In [ ]:
# Remove invalid data points according to Note1.1 and Note1.3
invalid_ID = invalid_ID_1 + invalid_ID_2
for i in range(15):
    for j in invalid_ID:
        Data.stage1["Measurement%d"%(i)] = Data.stage1["Measurement%d"%(i)][Data.stage1["Measurement%d"%(i)]['ID']!=j]

In [ ]:
# Remove invalid data points during start up period (<60s) at stage2
for i in range(15):
    Data.stage2["Measurement%d"%(i)] = Data.stage2["Measurement%d"%(i)][Data.stage2["Measurement%d"%(i)]['ID']>60]

In [ ]:
# Check Outputs Plot
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.stage1["Measurement%d"%(i)]['ID'],Data.stage1["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual')
    ax[i].plot(Data.stage1["Measurement%d"%(i)]['ID'],Data.stage1["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Setpoint')
    ax[i].set_ylabel('Stage1.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.stage2["Measurement%d"%(i)]['ID'],Data.stage2["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual')
    ax[i].plot(Data.stage2["Measurement%d"%(i)]['ID'],Data.stage2["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Setpoint')
    ax[i].set_ylabel('Stage2.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');

In [ ]:
# Smoothing noise (assumed those are random noise) with 60s window
Data.stage1_smooth = Data.stage1.copy()
Data.stage2_smooth = Data.stage2.copy()
for i in range(15):    
    Data.stage1_smooth["Measurement%d"%(i)] = Data.stage1_smooth["Measurement%d"%(i)].set_index(pd.DatetimeIndex(Data.stage1_smooth["Measurement%d"%(i)]['time_stamp']))
    Data.stage1_smooth["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)] = Data.stage1_smooth["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)].rolling('30s').mean()
    Data.stage2_smooth["Measurement%d"%(i)] = Data.stage2_smooth["Measurement%d"%(i)].set_index(pd.DatetimeIndex(Data.stage2_smooth["Measurement%d"%(i)]['time_stamp']))
    Data.stage2_smooth["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)] = Data.stage2_smooth["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)].rolling('30s').mean()

<a id='smooth noise'></a>

In [ ]:
# Check Outputs Plot after smoothing
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.stage1["Measurement%d"%(i)]['ID'],Data.stage1["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Measurement before Smoothing')
    ax[i].plot(Data.stage1["Measurement%d"%(i)]['ID'],Data.stage1["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Setpoint')
    ax[i].plot(Data.stage1_smooth["Measurement%d"%(i)]['ID'],Data.stage1_smooth["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Actual Measurement after Smoothing(30s)')
    ax[i].set_ylabel('Stage1.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');


In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(Data.stage2["Measurement%d"%(i)]['ID'],Data.stage2["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Measurement before Smoothing')
    ax[i].plot(Data.stage2["Measurement%d"%(i)]['ID'],Data.stage2["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Setpoint'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Setpoint')
    ax[i].plot(Data.stage2_smooth["Measurement%d"%(i)]['ID'],Data.stage2_smooth["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)],
            marker='o', markersize=0.5, linestyle=':', label='Actual Measurement after Smoothing(30s)')
    ax[i].set_ylabel('Stage2.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Time (Sec)')
    ax[i].legend(loc='upper right');

In [ ]:
for i in range(15): 
    
    Data.stage1_smooth["Measurement%d"%(i)] = Data.stage1_smooth["Measurement%d"%(i)][Data.stage1_smooth["Measurement%d"%(i)]['Stage1.Output.Measurement%d.U.Actual'% (i)].notnull()]
    Data.stage2_smooth["Measurement%d"%(i)] = Data.stage2_smooth["Measurement%d"%(i)][Data.stage2_smooth["Measurement%d"%(i)]['Stage2.Output.Measurement%d.U.Actual'% (i)].notnull()]
    

<a id='Variable type'></a>
#### 2.3 Check variable type
Independent variables are all continous numerical variable, no need to do extra processing. 

<a id='Scaling'></a>
#### 2.4 Scaling

In [ ]:
# Check range of input variables
Data.stage1_smooth["Measurement13"][C_cols_Stage2].describe() 
# Scales differs a lot among input variables
# However, input variables don't vary a lot, which won't provide rich info to the models

##### Note2: Summary from the input variables descriptive info above

1.Scales differs a lot among input variables.

2.Normalize features is important while comparing measurements that have different units, also a general requirement for many machine learning algorithms.

3.However, based on use cases, it might not make sense to normalize features.

4.Also, it's inproper to do normalization without background knowledge of input variables.

5.Therefore, these input variables would not be normalized.

<a id='Multi-collinearity'></a>
#### 2.5 Multi-collinearity

In [ ]:
C_cols_Stage1_smallVif=vif_cal(Data.stage1_smooth["Measurement0"][C_cols_Stage1]) 
# There is not severe multi-collinearity problem among the control input variables for stage1
# Although "Machine3.MotorRPM.C.Actual" might get a bit higher value of VIF, which should be noticed while modeling
# Rule of thumb: VIF >10, high multi-collinearity

In [ ]:
C_cols_Stage2_smallVif = vif_cal(Data.stage2_smooth["Measurement0"][C_cols_Stage2])
# There is severe multi-collinearity problem among the control input variables for stage2, which should be noticed while modeling
# Rule of thumb: VIF >10, high multi-collinearity

<a id='Feature transformation'></a>
#### 2.6 Feature Transformation
Not able to do feature transformation without background knowledge of input variables.

<a id='3.Finalizing Training Dataset'></a>
### 3. Finalizing Training Dataset

In [ ]:
# Input Variables: C_cols_Stage1 / C_cols_Stage1_smallVif, C_cols_Stage2 / C_cols_Stage2_smallVif

In [ ]:
for i in range(15):
    Data.stage1_smooth["Measurement%d"%(i)].reset_index(drop=True, inplace=True)
    Data.stage2_smooth["Measurement%d"%(i)].reset_index(drop=True, inplace=True)

In [ ]:
# Output Variables: Stage1.Measurement%d.DV_ErrorRate / Stage1.Output.Measurement%d.U.Actual, Stage2.Measurement%d.DV_ErrorRate / Stage2.Output.Measurement%d.U.Actual
for i in range(15):
    Data.stage1_smooth["Measurement%d"%(i)]["Stage1.Measurement%d.DV_ErrorRate"%(i)] = error_rate(Data.stage1_smooth["Measurement%d"%(i)],"Stage1.Output.Measurement%d.U.Actual"%(i),"Stage1.Output.Measurement%d.U.Setpoint"%(i))
    Data.stage2_smooth["Measurement%d"%(i)]["Stage2.Measurement%d.DV_ErrorRate"%(i)] = error_rate(Data.stage2_smooth["Measurement%d"%(i)],"Stage2.Output.Measurement%d.U.Actual"%(i),"Stage2.Output.Measurement%d.U.Setpoint"%(i))


In [ ]:
TrainingSet_stage1 = {} 
TrainingSet_stage2 = {} 
for i in range(15):
    df1=Data.stage1_smooth["Measurement%d"%(i)][["Stage1.Measurement%d.DV_ErrorRate"%(i),"Stage1.Output.Measurement%d.U.Actual"%(i)]]
    
    df2 =Data.stage1_smooth["Measurement%d"%(i)][C_cols_Stage1]
    df3=Data.stage2_smooth["Measurement%d"%(i)][["Stage2.Measurement%d.DV_ErrorRate"%(i),"Stage2.Output.Measurement%d.U.Actual"%(i)]]
    df4 = Data.stage2_smooth["Measurement%d"%(i)][C_cols_Stage2]
    TrainingSet_stage1["Measurement%d"%(i)]=pd.concat([df1, df2], axis=1)
    TrainingSet_stage2["Measurement%d"%(i)]=pd.concat([df3, df4], axis=1)
    
    TrainingSet_stage1["Measurement%d"%(i)].to_csv('TrainingSet_stage1_Measurement%d.csv'%(i),index=False)
    TrainingSet_stage2["Measurement%d"%(i)].to_csv('TrainingSet_stage2_Measurement%d.csv'%(i),index=False)

<a id='4.Modeling and Evaluation'></a>
### 4. Modeling and Evaluation

#### 4.1 Train-test data split

In [ ]:
X_train = {}
X_test = {}
y_train = {}
y_test = {}
X = {}
y = {}
for i in range(15):
    X["stage1Measurement%d"%(i)] = TrainingSet_stage1["Measurement%d"%(i)][C_cols_Stage1].values
    y["stage1Measurement%d"%(i)] = TrainingSet_stage1["Measurement%d"%(i)]["Stage1.Output.Measurement%d.U.Actual"%(i)].values
    X_train["stage1Measurement%d"%(i)],X_test["stage1Measurement%d"%(i)],y_train["stage1Measurement%d"%(i)],y_test["stage1Measurement%d"%(i)] = train_test_split(X["stage1Measurement%d"%(i)],y["stage1Measurement%d"%(i)],test_size=0.2,random_state=0)
    
    X["stage2Measurement%d"%(i)] = TrainingSet_stage2["Measurement%d"%(i)][C_cols_Stage2].values
    y["stage2Measurement%d"%(i)] = TrainingSet_stage2["Measurement%d"%(i)]["Stage2.Output.Measurement%d.U.Actual"%(i)].values
    X_train["stage2Measurement%d"%(i)],X_test["stage2Measurement%d"%(i)],y_train["stage2Measurement%d"%(i)],y_test["stage2Measurement%d"%(i)] = train_test_split(X["stage2Measurement%d"%(i)],y["stage2Measurement%d"%(i)],test_size=0.2,random_state=0)
    
       

#### 4.2 Linear Regression 

In [ ]:
lr_stage1 = {} # Model
lr_stage2 = {}
coeff_lr_stage1 = {} # Coefficient
coeff_lr_stage2 = {}
for i in range(15):
    lr_stage1["Measurement%d"%(i)] = LinearRegression()
    lr_stage1["Measurement%d"%(i)].fit(X_train["stage1Measurement%d"%(i)], y_train["stage1Measurement%d"%(i)])
    
    lr_stage2["Measurement%d"%(i)] = LinearRegression()
    lr_stage2["Measurement%d"%(i)].fit(X_train["stage2Measurement%d"%(i)], y_train["stage2Measurement%d"%(i)])
    
    coeff_lr_stage1["Measurement%d"%(i)] = pd.DataFrame(lr_stage1["Measurement%d"%(i)].coef_, TrainingSet_stage1["Measurement%d"%(i)][C_cols_Stage1].columns, columns=['Coefficient']) 
    coeff_lr_stage2["Measurement%d"%(i)] = pd.DataFrame(lr_stage2["Measurement%d"%(i)].coef_, TrainingSet_stage2["Measurement%d"%(i)][C_cols_Stage2].columns, columns=['Coefficient'])  


In [ ]:
y_pred_stage1 = {} # Predicted value
y_pred_stage2 = {}
y_actual_pred_stage1 = {} # Compare Actual value and Predicted value
y_actual_pred_stage2 = {}

for i in range(15):
    y_pred_stage1["Measurement%d"%(i)] = lr_stage1["Measurement%d"%(i)].predict(X_test["stage1Measurement%d"%(i)])
    y_actual_pred_stage1["Measurement%d"%(i)] = pd.DataFrame({'Actual': y_test["stage1Measurement%d"%(i)], 'Predicted': y_pred_stage1["Measurement%d"%(i)]})

    y_pred_stage2["Measurement%d"%(i)] = lr_stage2["Measurement%d"%(i)].predict(X_test["stage2Measurement%d"%(i)])
    y_actual_pred_stage2["Measurement%d"%(i)] = pd.DataFrame({'Actual': y_test["stage2Measurement%d"%(i)], 'Predicted': y_pred_stage2["Measurement%d"%(i)]})


In [ ]:
# Compare Actual test value and Predicted value at stage 1
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(y_actual_pred_stage1["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage1["Measurement%d"%(i)]['Actual'],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Test Measurement')
    ax[i].plot(y_actual_pred_stage1["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage1["Measurement%d"%(i)]['Predicted'],
            marker='o', markersize=0.5, linestyle=':', label='Predicted Measurement')
    ax[i].set_ylabel('Stage1.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Test Data Points')
    ax[i].legend(loc='upper right');


In [ ]:
# Compare Actual test value and Predicted value at stage 2
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(y_actual_pred_stage2["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage2["Measurement%d"%(i)]['Actual'],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Test Measurement')
    ax[i].plot(y_actual_pred_stage2["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage2["Measurement%d"%(i)]['Predicted'],
            marker='o', markersize=0.5, linestyle=':', label='Predicted Measurement')
    ax[i].set_ylabel('Stage2.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Test Data Points')
    ax[i].legend(loc='upper right');

In [ ]:
# Model performance: Mean Absolute Error, Mean Squared Error, Root Mean Squared Error

mean_absolute_error_stage1 = {}
mean_absolute_error_stage2 = {}
mean_squared_error_stage1 = {}
mean_squared_error_stage2 = {}
root_mean_squared_error_stage1 = {}
root_mean_squared_error_stage2 = {}

for i in range(15):
    mean_absolute_error_stage1["Measurement%d"%(i)] = metrics.mean_absolute_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)])
    mean_absolute_error_stage2["Measurement%d"%(i)] = metrics.mean_absolute_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)])
    
    mean_squared_error_stage1["Measurement%d"%(i)] = metrics.mean_squared_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)])
    mean_squared_error_stage2["Measurement%d"%(i)] = metrics.mean_squared_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)])
    
    root_mean_squared_error_stage1["Measurement%d"%(i)] = np.sqrt(metrics.mean_squared_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)]))
    root_mean_squared_error_stage2["Measurement%d"%(i)] = np.sqrt(metrics.mean_squared_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)]))


In [ ]:
# Transfer to Dataframe
mean_absolute_error_stage1_df = pd.DataFrame.from_dict(list(mean_absolute_error_stage1.items()))
mean_absolute_error_stage1_df.columns = ['Measurement', 'mean_absolute_error_stage1']
mean_squared_error_stage1_df = pd.DataFrame.from_dict(list(mean_squared_error_stage1.items()))
mean_squared_error_stage1_df.columns = ['Measurement', 'mean_squared_error_stage1']
root_mean_squared_error_stage1_df = pd.DataFrame.from_dict(list(root_mean_squared_error_stage1.items()))
root_mean_squared_error_stage1_df.columns = ['Measurement', 'root_mean_squared_error_stage1']

Performance_stage1 = pd.concat([mean_absolute_error_stage1_df, mean_squared_error_stage1_df,root_mean_squared_error_stage1_df], axis=1)
Performance_stage1 = Performance_stage1.loc[:,~Performance_stage1.columns.duplicated()]

mean_absolute_error_stage2_df = pd.DataFrame.from_dict(list(mean_absolute_error_stage2.items()))
mean_absolute_error_stage2_df.columns = ['Measurement', 'mean_absolute_error_stage2']
mean_squared_error_stage2_df = pd.DataFrame.from_dict(list(mean_squared_error_stage2.items()))
mean_squared_error_stage2_df.columns = ['Measurement', 'mean_squared_error_stage2']
root_mean_squared_error_stage2_df = pd.DataFrame.from_dict(list(root_mean_squared_error_stage2.items()))
root_mean_squared_error_stage2_df.columns = ['Measurement', 'root_mean_squared_error_stage2']

Performance_stage2 = pd.concat([mean_absolute_error_stage2_df, mean_squared_error_stage2_df,root_mean_squared_error_stage2_df], axis=1)
Performance_stage2 = Performance_stage2.loc[:,~Performance_stage2.columns.duplicated()]

Performance = pd.concat([Performance_stage1, Performance_stage2], axis=1)
Performance = Performance.loc[:,~Performance.columns.duplicated()]

Performance


<a id='lr performance'></a>

In [ ]:

ax = Performance.plot.bar(x=['Measurement'], y=['root_mean_squared_error_stage1', 'root_mean_squared_error_stage2'],
                          figsize=(30,10),fontsize=18)
ax.set_xlabel('Measurement',fontsize=18)
ax.legend(loc='upper right',fontsize=18)
plt.title('Linear Regression Model Performance',fontsize=18)

In [ ]:
#### R-square
lr_r2 = lr_stage1["Measurement0"].score(X_train["stage1Measurement0"], y_train["stage1Measurement0"])
print('R^2: {0}'.format(lr_r2))

In [ ]:
residuals = abs(y_test["stage1Measurement0"])-abs( y_pred_stage1["Measurement0"])
df_results = pd.DataFrame({'Actual': y_test["stage1Measurement0"], 'Predicted': y_pred_stage1["Measurement0"]})
df_results['Residuals']=residuals

In [ ]:
#### Assumption 1: linearity
def linear_assumption(df_results):
    """
    Linearity: Assumes that there is a linear relationship between the predictors and
               the response variable. If not, either a quadratic term or another
               algorithm should be used.
    """
    print('Assumption 1: Linear Relationship between the Target and the Feature', '\n')
        
    print('Checking with a scatter plot of actual vs. predicted.',
           'Predictions should follow the diagonal line.')
    
    # Plotting the actual vs predicted values
    sns.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False, size=7)
        
    # Plotting the diagonal line
    line_coords = np.arange(12, 13)
    plt.plot(line_coords, line_coords,  # X and y points
             color='darkorange', linestyle='--')
    plt.title('Actual vs. Predicted')
    plt.show()
linear_assumption(df_results)

In [ ]:
#### Assumption 2: Normality
def normal_errors_assumption(residuals, p_value_thresh=0.05):
    """
    Normality: Assumes that the error terms are normally distributed. If they are not,
    nonlinear transformations of variables may solve this.
               
    This assumption being violated primarily causes issues with the confidence intervals
    """
    from statsmodels.stats.diagnostic import normal_ad
    print('Assumption 2: The error terms are normally distributed', '\n')
    
    print('Using the Anderson-Darling test for normal distribution')

    # Performing the test on the residuals
    p_value = normal_ad(residuals)[1]
    print('p-value from the test - below 0.05 generally means non-normal:', p_value)
    
    # Reporting the normality of the residuals
    if p_value < p_value_thresh:
        print('Residuals are not normally distributed')
    else:
        print('Residuals are normally distributed')
    
    # Plotting the residuals distribution
    plt.subplots(figsize=(12, 6))
    plt.title('Distribution of Residuals')
    sns.distplot(residuals)
    plt.show()
    
    print()
    if p_value > p_value_thresh:
        print('Assumption satisfied')
    else:
        print('Assumption not satisfied')
        print()
        print('Confidence intervals will likely be affected')
        print('Try performing nonlinear transformations on variables')
normal_errors_assumption(residuals, p_value_thresh=0.05)

In [ ]:
#### Assumption 3: autocorrelation
def autocorrelation_assumption(df_results):
    """
    Autocorrelation: Assumes that there is no autocorrelation in the residuals. If there is
                     autocorrelation, then there is a pattern that is not explained due to
                     the current value being dependent on the previous value.
                     This may be resolved by adding a lag variable of either the dependent
                     variable or some of the predictors.
    """
    from statsmodels.stats.stattools import durbin_watson
    print('Assumption 3: No Autocorrelation', '\n')
    
    # Calculating residuals for the Durbin Watson-tests

    print('\nPerforming Durbin-Watson Test')
    print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
    print('0 to 2< is positive autocorrelation')
    print('>2 to 4 is negative autocorrelation')
    print('-------------------------------------')
    durbinWatson = durbin_watson(df_results['Residuals'])
    print('Durbin-Watson:', durbinWatson)
    if durbinWatson < 1.5:
        print('Signs of positive autocorrelation', '\n')
        print('Assumption not satisfied')
    elif durbinWatson > 2.5:
        print('Signs of negative autocorrelation', '\n')
        print('Assumption not satisfied')
    else:
        print('Little to no autocorrelation', '\n')
        print('Assumption satisfied')
autocorrelation_assumption(df_results)

In [ ]:
#### Assumption 4: Homoscedastricity
def homoscedasticity_assumption(df_results):
    """
    Homoscedasticity: Assumes that the errors exhibit constant variance
    """
    print('Assumption 4: Homoscedasticity of Error Terms', '\n')
    
    print('Residuals should have relative constant variance')
        
    
    # Plotting the residuals
    plt.subplots(figsize=(12, 6))
    ax = plt.subplot(111)  # To remove spines
    plt.scatter(x=df_results.index, y=df_results.Residuals, alpha=0.5)
    plt.plot(np.repeat(0, df_results.index.max()), color='darkorange', linestyle='--')
    ax.spines['right'].set_visible(False)  # Removing the right spine
    ax.spines['top'].set_visible(False)  # Removing the top spine
    plt.title('Residuals')
    plt.show()  
homoscedasticity_assumption(df_results)

#### 4.2 Bayesian Ridge Regression

In [ ]:
brr_stage1 = {} # Model
brr_stage2 = {}
coeff_brr_stage1 = {} # Coefficient
coeff_brr_stage2 = {}
for i in range(15):
    brr_stage1["Measurement%d"%(i)] = BayesianRidge(compute_score=True)
    brr_stage1["Measurement%d"%(i)].fit(X_train["stage1Measurement%d"%(i)], y_train["stage1Measurement%d"%(i)])
    
    brr_stage2["Measurement%d"%(i)] = BayesianRidge(compute_score=True)
    brr_stage2["Measurement%d"%(i)].fit(X_train["stage2Measurement%d"%(i)], y_train["stage2Measurement%d"%(i)])
    
    coeff_brr_stage1["Measurement%d"%(i)] = pd.DataFrame(brr_stage1["Measurement%d"%(i)].coef_, TrainingSet_stage1["Measurement%d"%(i)][C_cols_Stage1].columns, columns=['Coefficient']) 
    coeff_brr_stage2["Measurement%d"%(i)] = pd.DataFrame(brr_stage2["Measurement%d"%(i)].coef_, TrainingSet_stage2["Measurement%d"%(i)][C_cols_Stage2].columns, columns=['Coefficient'])  


In [ ]:
y_pred_stage1 = {} # Predicted value
y_pred_stage2 = {}
y_actual_pred_stage1 = {} # Compare Actual value and Predicted value
y_actual_pred_stage2 = {}

for i in range(15):
    y_pred_stage1["Measurement%d"%(i)] = brr_stage1["Measurement%d"%(i)].predict(X_test["stage1Measurement%d"%(i)])
    y_actual_pred_stage1["Measurement%d"%(i)] = pd.DataFrame({'Actual': y_test["stage1Measurement%d"%(i)], 'Predicted': y_pred_stage1["Measurement%d"%(i)]})

    y_pred_stage2["Measurement%d"%(i)] = brr_stage2["Measurement%d"%(i)].predict(X_test["stage2Measurement%d"%(i)])
    y_actual_pred_stage2["Measurement%d"%(i)] = pd.DataFrame({'Actual': y_test["stage2Measurement%d"%(i)], 'Predicted': y_pred_stage2["Measurement%d"%(i)]})


In [ ]:
# Compare Actual test value and Predicted value at stage 1
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(y_actual_pred_stage1["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage1["Measurement%d"%(i)]['Actual'],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Test Measurement')
    ax[i].plot(y_actual_pred_stage1["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage1["Measurement%d"%(i)]['Predicted'],
            marker='o', markersize=0.5, linestyle=':', label='Predicted Measurement')
    ax[i].set_ylabel('Stage1.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Test Data Points')
    ax[i].legend(loc='upper right');

In [ ]:
# Compare Actual test value and Predicted value at stage 2
fig = plt.figure()
fig, ax = plt.subplots(15,1,figsize=(30,150))

for i in range(15):
    ax[i].plot(y_actual_pred_stage2["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage2["Measurement%d"%(i)]['Actual'],
               marker='.', linestyle=':', linewidth=0.5, label='Actual Test Measurement')
    ax[i].plot(y_actual_pred_stage2["Measurement%d"%(i)].index.tolist(),y_actual_pred_stage2["Measurement%d"%(i)]['Predicted'],
            marker='o', markersize=0.5, linestyle=':', label='Predicted Measurement')
    ax[i].set_ylabel('Stage2.Output.Measurement%d'% (i))
    ax[i].set_xlabel('Test Data Points')
    ax[i].legend(loc='upper right');

In [ ]:
# Model performance: Mean Absolute Error, Mean Squared Error, Root Mean Squared Error

mean_absolute_error_stage1 = {}
mean_absolute_error_stage2 = {}
mean_squared_error_stage1 = {}
mean_squared_error_stage2 = {}
root_mean_squared_error_stage1 = {}
root_mean_squared_error_stage2 = {}

for i in range(15):
    mean_absolute_error_stage1["Measurement%d"%(i)] = metrics.mean_absolute_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)])
    mean_absolute_error_stage2["Measurement%d"%(i)] = metrics.mean_absolute_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)])
    
    mean_squared_error_stage1["Measurement%d"%(i)] = metrics.mean_squared_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)])
    mean_squared_error_stage2["Measurement%d"%(i)] = metrics.mean_squared_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)])
    
    root_mean_squared_error_stage1["Measurement%d"%(i)] = np.sqrt(metrics.mean_squared_error(y_test["stage1Measurement%d"%(i)], y_pred_stage1["Measurement%d"%(i)]))
    root_mean_squared_error_stage2["Measurement%d"%(i)] = np.sqrt(metrics.mean_squared_error(y_test["stage2Measurement%d"%(i)], y_pred_stage2["Measurement%d"%(i)]))


In [ ]:
# Transfer to Dataframe
mean_absolute_error_stage1_df = pd.DataFrame.from_dict(list(mean_absolute_error_stage1.items()))
mean_absolute_error_stage1_df.columns = ['Measurement', 'mean_absolute_error_stage1']
mean_squared_error_stage1_df = pd.DataFrame.from_dict(list(mean_squared_error_stage1.items()))
mean_squared_error_stage1_df.columns = ['Measurement', 'mean_squared_error_stage1']
root_mean_squared_error_stage1_df = pd.DataFrame.from_dict(list(root_mean_squared_error_stage1.items()))
root_mean_squared_error_stage1_df.columns = ['Measurement', 'root_mean_squared_error_stage1']

Performance_stage1 = pd.concat([mean_absolute_error_stage1_df, mean_squared_error_stage1_df,root_mean_squared_error_stage1_df], axis=1)
Performance_stage1 = Performance_stage1.loc[:,~Performance_stage1.columns.duplicated()]

mean_absolute_error_stage2_df = pd.DataFrame.from_dict(list(mean_absolute_error_stage2.items()))
mean_absolute_error_stage2_df.columns = ['Measurement', 'mean_absolute_error_stage2']
mean_squared_error_stage2_df = pd.DataFrame.from_dict(list(mean_squared_error_stage2.items()))
mean_squared_error_stage2_df.columns = ['Measurement', 'mean_squared_error_stage2']
root_mean_squared_error_stage2_df = pd.DataFrame.from_dict(list(root_mean_squared_error_stage2.items()))
root_mean_squared_error_stage2_df.columns = ['Measurement', 'root_mean_squared_error_stage2']

Performance_stage2 = pd.concat([mean_absolute_error_stage2_df, mean_squared_error_stage2_df,root_mean_squared_error_stage2_df], axis=1)
Performance_stage2 = Performance_stage2.loc[:,~Performance_stage2.columns.duplicated()]

Performance = pd.concat([Performance_stage1, Performance_stage2], axis=1)
Performance = Performance.loc[:,~Performance.columns.duplicated()]

Performance

<a id='brr performance'></a>

In [ ]:
ax = Performance.plot.bar(x=['Measurement'], y=['root_mean_squared_error_stage1', 'root_mean_squared_error_stage2'],
                          figsize=(30,10),fontsize=18)
ax.set_xlabel('Measurement',fontsize=18)
ax.legend(loc='upper right',fontsize=18)
plt.title('Bayesian Ridge Regression Model Performance',fontsize=18)

<a id='Coefficients'></a>

In [ ]:
# Performance of Bayesian Ridge Regression model is similar with performance of Linear Regression model 
# Bayesian Ridge Regression model give shrinkage to those coefficient of high VIF variables
# Take Measurement0 for example, see below

In [ ]:
# Coefficients of Linear Regression, eg: Measurement0
coeff_lr_stage1["Measurement0"]

In [ ]:
# Coefficients of Bayesian Ridge Regression, eg: Measurement0
coeff_brr_stage1["Measurement0"]

### save model

In [ ]:
for i in range(15):
    joblib.dump(lr_stage1["Measurement%d"%(i)],'lr_stage1_Measurement%d.pkl'%(i))
    joblib.dump(lr_stage2["Measurement%d"%(i)],'lr_stage2_Measurement%d.pkl'%(i))
    joblib.dump(brr_stage1["Measurement%d"%(i)],'brr_stage1_Measurement%d.pkl'%(i))
    joblib.dump(brr_stage2["Measurement%d"%(i)],'brr_stage2_Measurement%d.pkl'%(i))